In [1]:

from __future__ import annotations
from datetime import datetime, timezone
from syft_event import SyftEvents
from syft_rds.api.api import API
from syft_core import Client
from syft_rds.models.dataset import Dataset, DatasetCreate
from syft_rds.service.dataset_service import DatasetService
from syft_rds.connection.connection import get_connection
from syft_rds.service.context import BaseRPCContext
import shutil


In [2]:
client = Client.load()

In [3]:
# input policy: constrain the inputs of a syftfunction
# output policy: How do we modify the outputs of a syftfunction, how many outputs can you create

In [4]:
from syft_core import SyftBoxURL
from syft_rds.models.code import CodeCreate
from syft_rds.service.code_service import CodeService

box = SyftEvents("my-rds-app")

# TODO: pass this in the handler
context = BaseRPCContext(client=client, box=box)  


@box.on_request("/apis/list")
def get_apis() -> list[str]:
    """Respond to a ping request."""
    return {SyftBoxURL.from_path(k, client.workspace): v for k, v in box._SyftEvents__rpc.items()}

@box.on_request("/datasets/create")
def create_dataset(dataset: DatasetCreate) -> Dataset:
    """Respond to a ping request."""
    dataset_service = DatasetService.from_context(context)
    res = dataset_service.create_item(dataset)
    return res
    
@box.on_request("/datasets/list")
def list_datasets() -> list[Dataset]:
    """Respond to a ping request."""
    dataset_service = DatasetService.from_context(context)
    res = dataset_service.list_items()
    return res


@box.on_request("/code/create")
def create_code(code: CodeCreate) -> Code:
    """Respond to a ping request."""
    code_service = CodeService.from_context(context)
    res = code_service.create_item(code)
    return res


2025-02-11 11:07:31.415 | INFO     | syft_event.server2:register_rpc:132 - Register RPC: /apis/list
2025-02-11 11:07:31.419 | INFO     | syft_event.server2:register_rpc:132 - Register RPC: /datasets/create
2025-02-11 11:07:31.422 | INFO     | syft_event.server2:register_rpc:132 - Register RPC: /datasets/list
2025-02-11 11:07:31.424 | INFO     | syft_event.server2:register_rpc:132 - Register RPC: /code/create


In [5]:
app_data_dir = client.my_datasite / "apps" / context.box.app_name
shutil.rmtree(app_data_dir, ignore_errors=True)

In [6]:
conn = get_connection(box, mock=True)   

In [7]:
# DatasetCreate(name="my-dataset", description="my dataset").to_item()

In [8]:
res = conn.send(
    url=f"syft://{client.email}/api_data/my-rds-app/rpc/apis/list",
    body={},
    expiry="5m",
    cache=True,
)

In [9]:
api = API.from_email(client.email, conn)

In [10]:
api.datasets.create(body=DatasetCreate(name="my-dataset", description="my dataset"))

AttributeError: item_type

In [11]:
%debug

> /Users/koen/miniconda3/envs/syftbox/lib/python3.12/site-packages/pydantic/_internal/_model_construction.py(264)__getattr__()
    262             if private_attributes and item in private_attributes:
    263                 return private_attributes[item]
--> 264             raise AttributeError(item)
    265 
    266     @classmethod

*** NameError: name 'cls' is not defined
> /Users/koen/workspace/rds/syft-rds/src/syft_rds/service/dataset_service.py(11)from_context()
      9     @classmethod
     10     def from_context(cls, context: BaseRPCContext):
---> 11         return cls(db=DatasetQueryEngine.from_context(context, item_type=cls.item_type))
     12 
     13 

<class 'syft_rds.service.dataset_service.DatasetService'>


In [11]:
api.datasets.list()

[Dataset(uid=UUID('b598e97d-849a-406b-8ca3-e4875ea51670'), name='my-dataset', description='my dataset')]

# Appendix

In [12]:
res

{'syft://koen@openmined.org/api_data/my-rds-app/rpc/apis/list': <function __main__.get_apis() -> 'list[str]'>,
 'syft://koen@openmined.org/api_data/my-rds-app/rpc/datasets/create': <function __main__.create_dataset(dataset: 'DatasetCreate') -> 'Dataset'>,
 'syft://koen@openmined.org/api_data/my-rds-app/rpc/datasets/list': <function __main__.list_datasets() -> 'list[Dataset]'>,
 'syft://koen@openmined.org/api_data/my-rds-app/rpc/code/create': <function __main__.create_code(code: 'CodeCreate') -> 'Code'>}

In [13]:
res = conn.send(
    url=f"syft://{client.email}/api_data/my-rds-app/rpc/datasets/create",
    body=DatasetCreate(name="my-dataset", description="my dataset"),
    expiry="5m",
    cache=True,
)

In [14]:
res

Dataset(uid=UUID('2be4d154-c1b9-4ebf-b04d-87c4c11e4c8c'), name='my-dataset', description='my dataset')